In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [ ]:
nola = pd.read_csv('/Users/jmesmelican/Downloads/yelp_dataset/NOLA Restaurants.csv')

traindf = nola[nola['train']]
testdf = nola[~nola['train']]

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("gilf/english-yelp-sentiment")

model = TFAutoModelForSequenceClassification.from_pretrained("gilf/english-yelp-sentiment")

In [ ]:
splits = np.array_split(testdf['text'].replace(r'\n', ' ', regex=True),1000)

all_outputs = []

for i in range(len(splits)):
    print(i)
    tf_batch = tokenizer(
      list(splits[i]),
      padding=True,
      truncation=True,
      max_length=512,
      return_tensors="tf",
  )
    tf_outputs = model(tf_batch)
    all_outputs.append(tf_outputs.logits)
  

In [ ]:
logits = tf.concat(all_outputs,0)

In [ ]:
tf_predictions = tf.nn.softmax(logits, axis=-1)

In [ ]:
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
preds = tf_predictions.tolist()

In [ ]:
pred_classes = []

for i in range(len(preds)):
    pred_classes.append( preds[i].index(max(preds[i])) +1)

In [ ]:
a = np.array(pred_classes)
b = np.array(testdf['stars'])
acc = np.mean(a-b == 0)

In [ ]:
acc

In [ ]:
mae = np.mean(abs(np.matmul(np.array(preds),np.array([1,2,3,4,5])) - b))

In [ ]:
mae